In [1]:
import os
import pandas as pd
import plotly.express as px
import data_preprocessing.data_loading as dl

In [2]:
df = dl.get_party_positions_data(file_dir="data_folder", file_name='party_dataset.csv', country='Germany')

c:\Users\denid\Desktop\Case_Study_TUM25\data_preprocessing\data_loading.py:27: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


In [3]:
date_df = df[df['Calendar_Week'] == '202109']
date_df

,Country,Date,Calendar_Week,Party_Name,Foreign Special Relationships: Positive,Foreign Special Relationships: Negative,Anti-Imperialism,Military: Positive,Military: Negative,Peace,...,Labour Groups: Positive,Labour Groups: Negative,Middle Class and Professional Groups,Underprivileged Minority Groups,Non-economic Demographic Groups,Right-left position,Planned Economy,Market Economy,Welfare State,International Peace
89,Germany,26/09/2021,202109,90/Greens,0.101,0.0,0.278,0.633,0.734,0.785,...,3.747,0.000,0.025,0.177,0.152,-21.038,7.342,0.304,19.342,1.519
90,Germany,26/09/2021,202109,LINKE,0.021,0.0,0.412,0.021,3.376,1.091,...,8.481,0.000,0.000,0.000,0.103,-36.167,8.399,0.062,28.427,4.467
91,Germany,26/09/2021,202109,SPD,0.059,0.0,0.238,0.951,1.011,1.070,...,9.096,0.000,0.000,0.119,0.476,-24.673,5.351,0.119,22.414,2.081
92,Germany,26/09/2021,202109,FDP,0.222,0.0,0.310,1.329,0.443,0.576,...,3.057,0.266,0.842,0.000,0.133,0.266,3.057,6.424,14.222,1.019
93,Germany,26/09/2021,202109,CDU/CSU,0.252,0.0,0.000,3.279,0.180,0.288,...,3.495,0.072,0.108,0.180,0.252,3.495,5.081,3.063,10.703,0.468
94,Germany,26/09/2021,202109,SSW,0.000,0.0,0.293,0.220,0.147,0.514,...,6.676,0.000,0.147,3.155,0.073,-22.450,3.742,0.073,25.092,0.660
95,Germany,26/09/2021,202109,AfD,0.000,0.0,0.127,2.478,0.445,0.127,...,1.398,0.127,0.445,0.445,0.318,26.048,2.541,8.386,7.687,0.572


In [4]:
fig = px.scatter(date_df,
    x='Right-left position',
    y='Planned Economy',
    color='Party_Name'
    )
fig.update_traces(marker=dict(size=12))